# Standard Name Table

A so-called "standard name table" defines "standard names", which is a concept used by the [CF Convention](https://cfconventions.org/).

Those standard names are used to define the meaning of a numerical variable in files (typically netCDF4 files).

With this library, we can describe a standard name table using JSON-LD. **Note**, that only a simplified version of the original CF Conventions is modelled!

This notebook walks you through the main steps of building such a table yourself using Python:

In [1]:
import ssnolib
from ssnolib.namespace import SSNO
from ssnolib.prov import Person, Organization, Attribution
from ontolutils.namespacelib.m4i import M4I

Let's start by instantiate a table. We add a title and one or multiple associated "agents", which can be persons or organizations:

In [2]:
# Create to "Agents", which are Persons in this case:
agent1 = ssnolib.Person(
    firstName="Matthias",
    lastName="Probst",
    orcidId="https://orcid.org/0000-0001-8729-0482",
    id="https://orcid.org/0000-0001-8729-0482"
)
# Agent 2 is affiliated with an organization:
orga1 = ssnolib.Organization(name="Awesome Institute")
agent2 = ssnolib.Person(
    firstName="John",
    lastName="Doe",
    mbox="john@doe.com",
    affiliation=orga1
)

# instantiate the table:
snt = ssnolib.StandardNameTable(
    title='SNT from scratch',
    description="A table defined as part of a tutorial",
    version='v1',
    qualifiedAttribution=[
        Attribution(agent=agent1, hadRole=M4I.ContactPerson),
        Attribution(agent=agent2, hadRole=M4I.Supervisor),
        Attribution(agent=orga1)
    ]
)

In [3]:
snt.to_html(folder="tmp")

WindowsPath('tmp/SNT from scratch.html')

In [4]:
print(snt.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "dcat": "http://www.w3.org/ns/dcat#",
        "dcterms": "http://purl.org/dc/terms/",
        "prov": "http://www.w3.org/ns/prov#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "foaf": "http://xmlns.com/foaf/0.1/",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "schema": "https://schema.org/"
    },
    "@type": "ssno:StandardNameTable",
    "dcterms:title": "SNT from scratch",
    "dcterms:description": "A table defined as part of a tutorial",
    "dcat:version": "v1",
    "prov:qualifiedAttribution": [
        {
            "@type": "prov:Attribution",
            "prov:agent": {
                "@type": "prov:Person",
                "foaf:firstName": "Matthias",
                "foaf:lastName": "Probst",
                "m4i:orcidId": "https://orcid.org/0000-0001-8729-0482",
                "@id": "https://orcid

Let's add some standard names to the table:

In [5]:
snt.standardNames = [
    ssnolib.StandardName(
        standard_name="air_density",
        description="The density of air",
        unit="kg/m^3"
    ),
    ssnolib.StandardName(
        standard_name="coordinate",
        description="The spatial coordinate vector.",
        unit="m"
    ),
    ssnolib.StandardName(
        standard_name="velocity",
        description="The velocity vector of an object or fluid.",
        unit="m/s"
    )
]

So far we only have two standard names. We can define modification rules, to build new, verified standard names. For example, "x_velocity" would be a reasonable new standard name for the table.

So let's define such a modification rule. We call it a `Qualification`. The one we would like to define should be used directly of an already existing standard name, e.g. "SSNO:AnyStandardName":

In [6]:
component = ssnolib.VectorQualification(
    name="component",
    hasValidValues=["x", "y", "z"],
    description="The component of a vector",
    before=SSNO.AnyStandardName
)

transformation = ssnolib.Transformation(
    name="C_derivative_of_X",
    description="derivative of X with respect to distance in the component direction, which may be x, y or z.",
    altersUnit="[X]/[C]",
    hasCharacter=[
        ssnolib.Character(character="X", associatedWith=SSNO.AnyStandardName),
        ssnolib.Character(character="C", associatedWith=component.id),
    ]
)

associatedWith: https://matthiasprobst.github.io/ssno#AnyStandardName
associatedWith: _:N80f15a545190416ab35897232c273b8a


Add it to the SNT:

In [7]:
snt.hasModifier = [component, transformation]

We can check standard name strings, whether they apply to the modification rule:

In [8]:
snt.verify_name("vertical_velocity")

False

In [9]:
snt.verify_name("x_velocity")

False

In [10]:
snt.verify_name("x_component")

False

Also, adding new standard names can go through a verification:

In [11]:
#snt.add_new_standard_name("x_coordinate", verify=True) # verify=False will just add the standard name and interpret it as a core standard name

## Export standard name tables
We can export to JSON-LD and also generate an HTML file:

Export the current version of the standard name table to JSON-LD:

In [12]:
print(snt.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "dcat": "http://www.w3.org/ns/dcat#",
        "dcterms": "http://purl.org/dc/terms/",
        "prov": "http://www.w3.org/ns/prov#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "foaf": "http://xmlns.com/foaf/0.1/",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "schema": "http://schema.org/",
        "skos": "http://www.w3.org/2004/02/skos/core#"
    },
    "@type": "ssno:StandardNameTable",
    "dcterms:title": "SNT from scratch",
    "dcterms:description": "A table defined as part of a tutorial",
    "dcat:version": "v1",
    "prov:qualifiedAttribution": [
        {
            "@type": "prov:Attribution",
            "prov:agent": {
                "@type": "prov:Person",
                "foaf:firstName": "Matthias",
                "foaf:lastName": "Probst",
                "m4i:orcidId": "https://orcid.org/0000

Write the HTML file:

In [13]:
snt.to_html(folder="tmp")

WindowsPath('tmp/SNT from scratch.html')

In [14]:
with open(f"tmp/{snt.title}.jsonld", "w", encoding="utf-8") as f:
    f.write(snt.model_dump_jsonld())

In [15]:
sntLoaded = ssnolib.StandardNameTable.parse(f"tmp/{snt.title}.jsonld")

associatedWith: _:N80f15a545190416ab35897232c273b8a
associatedWith: https://matthiasprobst.github.io/ssno#AnyStandardName
associatedWith: _:N80f15a545190416ab35897232c273b8a
associatedWith: https://matthiasprobst.github.io/ssno#AnyStandardName


In [16]:
sntLoaded.qualifiedAttribution[0].agent.model_dump(exclude_none=True)

{'id': '_:Na9a2f4791355454e989d0143e5748b94',
 'firstName': 'Matthias',
 'lastName': 'Probst',
 'orcidId': 'https://orcid.org/0000-0001-8729-0482',
 '@id': 'https://orcid.org/0000-0001-8729-0482',
 '@type': 'http://www.w3.org/ns/prov#Person'}

In [17]:
sntLoaded.hasModifier

[Qualification(id=_:N3cce5c0eabab4393a494329c9b561f40, name=component, description=The component of a vector, before=https://matthiasprobst.github.io/ssno#AnyStandardName, hasValidValues=[AnnotatedValue(id=_:Ncaac28c019e6476c87dacefaddaff3c5, value=x, annotation=No description available., type=https://matthiasprobst.github.io/ssno#AnnotatedValue), AnnotatedValue(id=_:N307a71c4ae7545968f0419fc25d00b18, value=z, annotation=No description available., type=https://matthiasprobst.github.io/ssno#AnnotatedValue), AnnotatedValue(id=_:N3dec12c771e64d57874bb979a3fa98fd, value=y, annotation=No description available., type=https://matthiasprobst.github.io/ssno#AnnotatedValue)], type=https://matthiasprobst.github.io/ssno#VectorQualification),
 Transformation(id=_:N9d7ffd3d8fc34b92b4062ac28f063c11, name=C_derivative_of_X, description=derivative of X with respect to distance in the component direction, which may be x, y or z., altersUnit=[X]/[C], hasCharacter=[Character(id=_:Nd31f471a320e49ca98893e4e

## From online resource

Let's pare the CF Convention, which is the model role for the library: [CF Convention table](http://cfconventions.org/Data/cf-standard-names/current/src/cf-standard-name-table.xml)

In [18]:
distribution = ssnolib.dcat.Distribution(
    title='XML Table',
    download_URL='http://cfconventions.org/Data/cf-standard-names/current/src/cf-standard-name-table.xml',
    media_type='application/xml'
)

In [19]:
snt = ssnolib.StandardNameTable(
    title='CF Standard Name Table (latest version)',
    distribution=distribution
)

In [20]:
print(snt.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "dcat": "http://www.w3.org/ns/dcat#",
        "dcterms": "http://purl.org/dc/terms/",
        "prov": "http://www.w3.org/ns/prov#",
        "ssno": "https://matthiasprobst.github.io/ssno#"
    },
    "@type": "ssno:StandardNameTable",
    "dcterms:title": "CF Standard Name Table (latest version)",
    "dcat:distribution": [
        {
            "@type": "dcat:Distribution",
            "dcterms:title": "XML Table",
            "dcat:downloadURL": "http://cfconventions.org/Data/cf-standard-names/current/src/cf-standard-name-table.xml",
            "dcat:mediaType": "https://www.iana.org/assignments/media-types/application/xml",
            "@id": "_:N580a5970c3ee44a889c696fd8aca7bfc"
        }
    ],
    "ssno:standardNames": [],
    "@id": "_:Nc6f02a00969e4d0293fd3f35ea646421"
}


Maybe we would like to get all the standard names. We can do this by calling `fetch()` or instantiate the standard name table using `parse()`:

In [21]:
from ontolutils import QUDT_UNIT

additional_qudts = {
    # other:
    'kg m-1 s-1': QUDT_UNIT.KiloGM_PER_M_SEC,
    'm-2 s-1': QUDT_UNIT.M2_PER_SEC,
    'K s': QUDT_UNIT.K_SEC,
    'W s m-2': QUDT_UNIT.W_SEC_PER_M2,
    'N m-1': QUDT_UNIT.N_PER_M,
    'mol mol-1': QUDT_UNIT.MOL_PER_MOL,
    'mol/mol': QUDT_UNIT.MOL_PER_MOL,
    'm4 s-1': QUDT_UNIT.M4_PER_SEC,
    'K Pa s-1': QUDT_UNIT.K_PA_PER_SEC,
    'Pa m s-1': QUDT_UNIT.PA_M_PER_SEC,
    'radian': QUDT_UNIT.RAD,
    'degree s-1': QUDT_UNIT.DEG_PER_SEC,
    'Pa m s-2': QUDT_UNIT.PA_M_PER_SEC2,
    'sr': QUDT_UNIT.SR,
    'sr-1': QUDT_UNIT.PER_SR,
    'm year-1': QUDT_UNIT.M_PER_YR,
    'mol m-2 s-1 sr-1': QUDT_UNIT.MOL_PER_M2_SEC_SR,
    'mol m-2 s-1 m-1 sr-1': QUDT_UNIT.MOL_PER_M2_SEC_M_SR,
    'Pa-1 s-1': QUDT_UNIT.PA_PER_SEC,
    'm-1 s-1': QUDT_UNIT.PER_M_SEC,
    'm2 s rad-1': QUDT_UNIT.M2_SEC_PER_RAD,
    'W/m2': QUDT_UNIT.W_PER_M2,
    'dbar': QUDT_UNIT.DeciBAR
}

In [22]:
snt = ssnolib.StandardNameTable.parse(snt.distribution[0], make_standard_names_lowercase=True, qudt_lookup=additional_qudts)

C:\Users\matth\Documents\GitHub\ssnolib\ssnolib\standard_name_table.py:416: UserWarning: Could not parse {'standardName': 'change_over_time_in_sea_water_salinity', 'unit': '1e-3', 'description': '"change_over_time_in_X" means change in a quantity X over a time-interval, which should be defined by the bounds of the time coordinate. Sea water salinity is the salt content of sea water, often on the Practical Salinity Scale of 1978. However, the unqualified term \'salinity\' is generic and does not necessarily imply any particular method of calculation. The units of salinity are dimensionless and the units attribute should normally be given as 1e-3 or 0.001 i.e. parts per thousand. There are standard names for the more precisely defined salinity quantities: sea_water_knudsen_salinity, S_K (used for salinity observations between 1901 and 1966), sea_water_cox_salinity, S_C (used for salinity observations between 1967 and 1977), sea_water_practical_salinity, S_P (used for salinity observation

In [23]:
snt.to_html(folder="tmp")

WindowsPath('tmp/cf-standard-name-table.html')

Write to JSON-LD file:

In [ ]:
with open(f"tmp/{snt.title}.jsonld", "w", encoding="utf-8") as f:
    f.write(snt.model_dump_jsonld())

Instantiate a Standard name table from a JSON-LD:

In [ ]:
ssnolib.StandardNameTable.parse(f"tmp/{snt.title}.jsonld")